## Useful Links
- http://www.cpcb.gov.in/CAAQM/frmUserAvgReportCriteria.aspx
- http://www.cpcb.gov.in/CAAQM/mapPage/frmindiamap.aspx

### Make Use of Beautiful Soup to automate fetching data
- Historic Annual PM2.5, PM10 Plot
    - INDIA - National
    - Delhi NCR
    - Delhi Location wise
    - Gurgaon

###Mechanize Links:
   - http://www.search.sourceforge.net/mechanize/
   - Forms Example: http://www.search.sourceforge.net/mechanize/forms.html
   - BSoup Input Parsing: http://stackoverflow.com/questions/23001678/python-beautiful-soup-form-input-parsing

In [1]:
from bs4 import BeautifulSoup
import requests
import mechanize
import re

In [2]:
url = 'http://www.cpcb.gov.in/CAAQM/frmUserAvgReportCriteria.aspx'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

print [(element['name']) for element in soup.find_all('input')]

[u'__VIEWSTATE', u'__EVENTVALIDATION', u'btnAdd', u'btnRemove', u'txtDateFrom', u'imgBtnFrom', u'txtDateTo', u'imgBtnTo', u'btnSubmit', u'btnCancel', u'btnClose']


In [3]:
br = mechanize.Browser()
br.set_handle_robots(False)
r1 = br.open(url)
# print r1.read()  # body
forms = mechanize.ParseResponse(r1, backwards_compat=False)
form = forms[0]
# print form

form["ddlState"] = ["1",]
r1 = br.open(form.click())
forms = mechanize.ParseResponse(r1, backwards_compat=False)
form = forms[0]

form["ddlCity"] = ["21",]
r1 = br.open(form.click())
forms = mechanize.ParseResponse(r1, backwards_compat=False)
form = forms[0]

form["ddlStation"] = ["96",]
r1 = br.open(form.click())
forms = mechanize.ParseResponse(r1, backwards_compat=False)
form = forms[0]

form["lstBoxChannelLeft"] = ["1715",]
r1 = br.open(form.click())
forms = mechanize.ParseResponse(r1, backwards_compat=False)
form = forms[0]

form["ddlCriteria"] = ["0",]
r1 = br.open(form.click())
forms = mechanize.ParseResponse(r1, backwards_compat=False)
form = forms[0]

form["txtYear"] = "2013"
form["txtYearTo"] = "2017"
r1 = br.open(form.click())
forms = mechanize.ParseResponse(r1, backwards_compat=False)
form = forms[0]

form["btnSubmit"] = "True"
r1 = br.open(form.click())
br.select_form(nr=0)
r1 = br.submit()

In [81]:
def getOptions(param, form):
    return re.findall("{}=\[(.+)\]".format(param), str(form))

def getData():
    br = mechanize.Browser()
    br.set_handle_robots(False)
    res = br.open(url)
    forms = mechanize.ParseResponse(res, backwards_compat=False)
    form = forms[0]
    form["ddlState"] = ["1",]
    form = updateForm(br, form)
    options = getOptions("ddlCity", form)
    print options, form
    return
    form["ddlCity"] = ["21",]
    form = updateForm(br, form)
    
    form["ddlStation"] = ["96",]
    form = updateForm(br, form)
    
    form["lstBoxChannelLeft"] = ["1715",]
    form = updateForm(br, form)
    
    form["ddlCriteria"] = ["0",]
    form = updateForm(br, form)
    
    form["txtYear"] = "2013"
    form["txtYearTo"] = "2017"
    form = updateForm(br, form)
    
    form["btnSubmit"] = "True"
    form = updateForm(br, form)
    
    br.select_form("form1")
    res = br.submit(name='btnSubmit')
    html = res.read()
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def updateForm(br, form):
    res = br.open(form.click())
    forms = mechanize.ParseResponse(res, backwards_compat=False)
    form = forms[0]
    return form

def getValsHtml(table):
    data = []
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values
    vals = re.findall("\d+\.\d+", data[0][0])[:-1]
    data = []
    for val in vals:
        yr = val[:4]
        data.append([yr, val[4:]])
    return data

In [82]:
getData()

['*Select City, 21, 9'] <form1 POST http://www.cpcb.gov.in/CAAQM/frmUserAvgReportCriteria.aspx application/x-www-form-urlencoded
  <HiddenControl(__VIEWSTATE=/wEPDwUJNDIxMTc4NDU4D2QWAgIDD2QWAgIDD2QWAgIDD2QWAmYPZBYcAgUPEA8WBh4NRGF0YVRleHRGaWVsZAUJU3RhdGVOYW1lHg5EYXRhVmFsdWVGaWVsZAUHU3RhdGVJRB4LXyFEYXRhQm91bmRnZBAVHwxTZWxlY3QgU3RhdGUOQW5kaHJhIFByYWRlc2gRQXJ1bmFjaGFsIFByYWRlc2gFQXNzYW0FQmloYXIMQ2hoYXR0aXNnYXJoBURlbGhpA0dvYQdHdWphcmF0B0hhcnlhbmEQSGltYWNoYWwgUHJhZGVzaBFKYW1tdSBhbmQgS2FzaG1pcglKaGFya2hhbmQJS2FybmF0YWthBktlcmFsYQ1NYWR5YSBQcmFkZXNoC01haGFyYXNodHJhB01hbmlwdXIJTWVnaGFsYXlhB01pem9yYW0ITmFnYWxhbmQGT3Jpc3NhBlB1bmphYglSYWphc3RoYW4GU2lra2ltClRhbWlsIE5hZHUHVHJpcHVyYQtVdHRhcmFuY2hhbA1VdHRhciBQcmFkZXNoC1dlc3QgQmVuZ2FsCVRlbGFuZ2FuYRUfDFNlbGVjdCBTdGF0ZQExATIBMwE0ATUBNgE3ATgBOQIxMAIxMQIxMgIxMwIxNAIxNQIxNgIxNwIxOAIxOQIyMAIyMQIyMgIyMwIyNAIyNQIyNgIyNwIyOAIyOQIzMBQrAx9nZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnFgECAWQCBw8QDxYGHwAFCENpdHlOYW1lHwEFBkNpdHlJRB8CZ2QQFQMLU2VsZWN0IENpdHkIVGlydXBhdGkNVm

In [16]:
soup = getData()
table = soup.find(id="gvReportStation")
data = getValsHtml(table)
print data

[[u'2016', u'21.87'], [u'2017', u'23.62']]


In [4]:
br.select_form("form1")
r1 = br.submit(name='btnSubmit')
html = r1.read()
soup = BeautifulSoup(html, 'html.parser')

In [6]:
vals = re.findall("\d+\.\d+", data[0][0])[:-1]
for val in vals:
    yr = val[:4]
    print yr, val[4:]

2016 21.87
2017 23.62
